<a href="https://colab.research.google.com/github/efandresena/SemEval/blob/main/subtask3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import random
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments, DataCollatorWithPadding,
    EarlyStoppingCallback
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')


# MULTILINGUAL

In [ ]:

workdir = "/content/drive/MyDrive/NLP/SemEval"

CONFIG = {
    'model': 'xlm-roberta-large',
    'max_len': 256,
    'epochs': 10,
    'lr': 3e-5,
    'batch': 16,
    'grad_accum': 2,
    'warmup': 0.1,
    'weight_decay': 0.01,
    'augment_factor': 3,
}

LABELS = ['stereotype', 'vilification', 'dehumanization', 'extreme_language', 'lack_of_empathy', 'invalidation']


SYNONYMS = {
    'eng': {
        'stereotype': {
            'typical': ['usual', 'common', 'standard', 'normal', 'predictable', 'stereotypical', 'cliché'],
            'always': ['constantly', 'invariably', 'forever', 'perpetually', 'without exception', 'eternally', 'endlessly'],
            'all': ['every', 'each', 'everyone', 'the whole', 'entirely', 'totally', 'universally'],
            'lazy': ['idle', 'inactive', 'slothful', 'indolent', 'shiftless', 'work-shy', 'sluggish', 'lethargic'],
            'stupid': ['dumb', 'foolish', 'idiotic', 'brainless', 'dim-witted', 'moronic', 'imbecilic', 'dense'],
            'criminal': ['lawbreaker', 'offender', 'thug', 'delinquent', 'felon', 'crook', 'gangster', 'outlaw'],
            'woke': ['progressive', 'liberal', 'sjw', 'virtue-signaler', 'politically correct', 'radical left'],
            'maga': ['trumpist', 'nationalist', 'populist', 'far-right', 'conservative extremist'],
            'illegal': ['undocumented', 'alien', 'invader', 'trespasser', 'unauthorized'],
            'white': ['caucasian', 'european', 'anglo', 'pale-skinned'],
            'red': ['republican', 'conservative', 'right-wing', 'gop'],
            'anti': ['against', 'opposed', 'hostile', 'contrary'],
            'left': ['liberal', 'democrat', 'progressive', 'socialist'],
            'mob': ['crowd', 'horde', 'gang', 'rabble', 'throng'],
            'radical': ['extreme', 'fanatic', 'militant', 'zealot'],
            'liberal': ['progressive', 'leftist', 'democrat', 'socialist'],
            'nazis': ['fascists', 'extremists', 'supremacists', 'neo-nazis']
        },
        'vilification': {
            'evil': ['wicked', 'sinister', 'malevolent', 'vile', 'malicious', 'diabolical', 'demonic'],
            'scum': ['filth', 'trash', 'garbage', 'dregs', 'riffraff', 'lowlife', 'dirt'],
            'vermin': ['pest', 'parasite', 'rat', 'insect', 'bug', 'cockroach', 'rodent'],
            'monster': ['beast', 'demon', 'fiend', 'ogre', 'brute', 'devil', 'savage'],
            'worthless': ['useless', 'valueless', 'good-for-nothing', 'pathetic', 'pitiful', 'no-good', 'trashy'],
            'xenophobia': ['racism', 'bigotry', 'prejudice', 'intolerance', 'discrimination'],
            'fascist': ['authoritarian', 'dictatorial', 'totalitarian', 'oppressive', 'tyrannical'],
            'cleansing': ['purging', 'elimination', 'eradication', 'extermination']
        },
        'dehumanization': {
            'animal': ['beast', 'creature', 'brute', 'pig', 'dog', 'rat', 'monkey'],
            'subhuman': ['less than human', 'primitive', 'inferior being', 'untermensch', 'barbaric', 'underdeveloped'],
            'savage': ['barbarian', 'wild', 'uncivilized', 'feral', 'brutish', 'heathen', 'primitive'],
            'breed': ['reproduce', 'spawn', 'multiply like rabbits', 'infest', 'proliferate', 'propagate'],
            'ethnic': ['racial', 'tribal', 'cultural', 'national'],
            'cleansing': ['purging', 'ethnic purge', 'genocide', 'expulsion'],
            'shields': ['human shields', 'cannon fodder', 'pawns', 'bait'],
            'genocide': ['mass murder', 'extermination', 'holocaust', 'slaughter']
        },
        'extreme_language': {
            'destroy': ['annihilate', 'obliterate', 'wipe out', 'crush', 'demolish', 'eradicate', 'decimate'],
            'exterminate': ['eliminate', 'eradicate', 'wipe off the map', 'root out', 'extinguish', 'liquidate'],
            'massacre': ['slaughter', 'butchery', 'carnage', 'bloodbath', 'genocide', 'pogrom'],
            'purge': ['cleanse', 'eliminate', 'rid the world of', 'expel', 'ethnically cleanse', 'remove'],
            'fuck': ['screw', 'damn', 'curse', 'eff'],
            'war': ['conflict', 'battle', 'invasion', 'aggression']
        },
        'lack_of_empathy': {
            'deserve': ['merit', 'earn', 'have it coming', 'brought it on themselves', 'asked for it', 'warranted'],
            'fault': ['blame', 'responsibility', 'their own doing', 'self-inflicted', 'guilty', 'culpable'],
            'who cares': ['so what', 'not my problem', 'big deal', 'whatever', 'cry me a river', 'tough'],
            'deal with it': ['accept it', 'cope', 'get over it', 'suck it up', 'tough luck', 'move on', 'endure'],
            'should': ['ought to', 'must', 'needs to'],
            'say': ['claim', 'allege', 'assert'],
            'never': ['not ever', 'no way', 'impossible'],
            'stop': ['halt', 'cease', 'quit'],
            'didnt': ["didn't", 'did not'],
            'cant': ["can't", 'cannot']
        },
        'invalidation': {
            'exaggerate': ['overstate', 'dramatize', 'blow out of proportion', 'make a mountain out of a molehill', 'overreact', 'hyperbolize'],
            'fake': ['false', 'fabricated', 'hoax', 'staged', 'made up', 'phony', 'bogus'],
            'lie': ['falsehood', 'untruth', 'deception', 'fabrication', 'misinformation', 'fib'],
            'pretending': ['faking', 'acting', 'playing the victim', 'crocodile tears', 'attention-seeking', 'simulating'],
            'rigged': ['fixed', 'manipulated', 'tampered', 'corrupt'],
            'stolen': ['robbed', 'taken', 'pilfered', 'swindled'],
            'news': ['fake news', 'propaganda', 'misinformation']
        }
    },
    'swa': {
        'stereotype': {
            'kawaida': ['ya kawaida', 'ya mara kwa mara', 'kwa kawaida', 'mara nyingi', 'kila wakati', 'kila mara'],
            'wote': ['kila mtu', 'watu wote', 'kila', 'wengine wote', 'jamii nzima', 'wengine'],
            'vizembe': ['wavivu', 'wazembe', 'wasio na bidii', 'walevi', 'wasiotaka kufanya kazi', 'wafala'],
            'wajinga': ['wapumbavu', 'wasio na akili', 'wapumbafu', 'wafala', 'wasio na busara', 'wazimu', 'wazembe'],
            'wakikuyu': ['wakikuyu', 'kabila la kikuyu', 'watu wa mlima'],
            'wajaluo': ['wajaluo', 'kabila la luo', 'watu wa ziwa'],
            'matako': ['makalio', 'tako', 'nyuma'],
            'mjinga': ['mpumbavu', 'mjinga', 'mzembe'],
            'wakisii': ['wakisii', 'kabila la kisii'],
            'wajinga': ['wapumbavu', 'wafala'],
            'wakamba': ['wakamba', 'kabila la kamba'],
            'waluhya': ['waluhya', 'kabila la luhya'],
            'wahindi': ['wahindi', 'waasia'],
            'waarabu': ['waarabu', 'waislamu']
        },
        'vilification': {
            'uovu': ['ubaya', 'ushaitani', 'ufisadi', 'maovu', 'hatari', 'maadui'],
            'takataka': ['uchafu', 'taka', 'matope', 'jazba', 'mazibo', 'mifugo', 'ufisadi'],
            'mnyama': ['wanyama', 'mkali', 'mwindaji', 'mla nyama', 'nyama mbichi', 'mshenzi'],
            'shetani': ['ibilisi', 'pepo mbaya', 'mlaghai', 'adui', 'mwuaji', 'mchawi'],
            'majembe': ['majembe', 'wachawi', 'walaghai'],
            'punda': ['punda', 'mpumbavu', 'mzembe']
        },
        'dehumanization': {
            'wanyama': ['viumbe', 'wakali', 'nyama', 'mbwa', 'panya', 'mende', 'mshenzi'],
            'viumbe': ['vidudu', 'vitu', 'wadudu', 'visumbufu', 'vimelea', 'vizibao', 'vichafu'],
            'wakali': ['wavamizi', 'wasio na ustaarabu', 'washenzi', 'waporaji', 'wanyang\'anyi', 'wauaji'],
            'punda': ['punda', 'mnyama', 'mpumbavu'],
            'nigga': ['nigga', 'mwafrika', 'mweusi']
        },
        'extreme_language': {
            'kuangamiza': ['kuharibu kabisa', 'kufuta', 'kumaliza', 'kuondoa milele', 'kufagia', 'kuchinja'],
            'kuchinja': ['kuua kwa wingi', 'mauaji', 'kuchinja damu', 'kumwaga damu', 'kufyeka', 'kuangamiza'],
            'kusafisha': ['kuondoa', 'kufuta', 'kufukuza', 'kusafisha jamii', 'kuondoa uchafu', 'kufagia'],
            'kutomba': ['kutombana', 'kufanya ngono', 'kula nyama'],
            'kuma': ['kuma', 'uume', 'ngono']
        },
        'lack_of_empathy': {
            'anastahili': ['anapaswa', 'ana haki', 'anastahiki', 'amejiletea', 'ni hatia yake', 'ameleta mwenyewe'],
            'lawama': ['hatia', 'kosa', 'makosa yao', 'wao wenyewe walitafuta', 'ni wao wenye lawama', 'wajilaumu'],
            'sina haja': ['sio shida yangu', 'hakuna shida', 'siwajali', 'nini shida', 'wafanye wenyewe', 'sina wakati'],
            'deal with it': ['vumilia', 'kabiliana nayo', 'shughulikia mwenyewe', 'kabiliana', 'vumilia tu', 'shinda nayo'],
            'mbaya': ['mbaya', 'baya', 'hatari']
        },
        'invalidation': {
            'kuviza': ['kuzidisha', 'kubana', 'kufanya drama', 'kuzusha', 'kuleta kelele', 'kuzidisha'],
            'uongo': ['udanganyifu', 'uwongo', 'hadaa', 'uzushi', 'habari za uongo', 'uwongo mtupu'],
            'kuigiza': ['kufanya mchezo', 'kudanganya', 'kujifanya', 'kujidai', 'kufanya victim', 'kujifanya mwathirika'],
            'huyu': ['huyu', 'hii', 'hawa'],
            'lol': ['lol', 'kicheko', 'dhihaka']
        }
    }
}

In [ ]:
import string
import random

def augment_text(text, category, lang, prob=0.8):
    cat_syns = SYNONYMS.get(lang, {}).get(category, {})
    if not cat_syns:
        return text
    words = text.split()
    aug_words = []
    changed = False
    for word in words:
        # Better cleaning: strip punctuation and lower for matching
        cleaned = word.translate(str.maketrans('', '', string.punctuation)).lower()
        if cleaned in cat_syns and random.random() < prob:
            syn = random.choice(cat_syns[cleaned])
            # Preserve original capitalization
            if word and word[0].isupper():
                syn = syn.capitalize()
            # Re-attach punctuation if original had it
            if word and word[-1] in string.punctuation:
                syn += word[-1]
            aug_words.append(syn)
            changed = True
        else:
            aug_words.append(word)
    new_text = ' '.join(aug_words)
    return new_text, changed  # Return if changed

def augment_df(df, lang, factor=3):
    print(f"Augmenting {lang}: {len(df)} samples")
    aug_texts = []
    aug_labels = []
    total_attempts = 0
    successful_augs = 0

    for _, row in df.iterrows():
        text = row['text']
        label_vals = {label: row[label] for label in LABELS}
        pos_labels = [l for l, v in label_vals.items() if v == 1]

        if not pos_labels:
            continue

        # Calculate how many augs per row (your original logic, but ensure >0 for rare)
        min_count = min(df[l].sum() for l in pos_labels)
        n_aug = max(3, factor) if min_count < 100 else factor  # Force at least some

        for _ in range(n_aug):
            total_attempts += 1
            cat = random.choice(pos_labels)
            aug_text, was_changed = augment_text(text, cat, lang, prob=0.9)  # Higher prob

            if was_changed:  # Only add if actual change
                aug_texts.append(aug_text)
                aug_labels.append(label_vals.copy())
                successful_augs += 1

    print(f"  Attempts: {total_attempts}, Successful changes: {successful_augs}")

    if aug_texts:
        aug_df = pd.DataFrame({'text': aug_texts, **{l: [d[l] for d in aug_labels] for l in LABELS}})
        result = pd.concat([df, aug_df], ignore_index=True)
        print(f"  +{len(aug_df)} augmented → {len(result)} total")
        return result

    print("  No successful augmentations generated.")
    return df

In [ ]:

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, pos_weight=None):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.pos_weight = pos_weight

    def forward(self, inputs, targets):
        bce = nn.functional.binary_cross_entropy_with_logits(inputs, targets, reduction='none', pos_weight=self.pos_weight)
        probs = torch.sigmoid(inputs)
        pt = torch.where(targets == 1, probs, 1 - probs)
        focal = (1 - pt) ** self.gamma
        if self.alpha is not None:
            alpha_w = torch.where(targets == 1, self.alpha, 1 - self.alpha)
            focal = alpha_w * focal
        return (focal * bce).mean()

class WeightedTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        pos_weight = self.class_weights.to(outputs.logits.device) if self.class_weights is not None else None
        loss = FocalLoss(alpha=0.25, gamma=2.0, pos_weight=pos_weight)(outputs.logits, labels)
        return (loss, outputs) if return_outputs else loss

class PolarizationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = self.tokenizer(self.texts[idx], truncation=True, max_length=self.max_len, padding=False, return_tensors='pt')
        item = {k: v.squeeze(0) for k, v in enc.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item


In [ ]:

def compute_metrics(pred):
    probs = torch.sigmoid(torch.from_numpy(pred.predictions)).numpy()
    preds = (probs > 0.5).astype(int)
    f1_macro = f1_score(pred.label_ids, preds, average='macro', zero_division=0)
    f1_per = f1_score(pred.label_ids, preds, average=None, zero_division=0)
    metrics = {'f1_macro': f1_macro}
    for i, label in enumerate(LABELS):
        metrics[f'f1_{label}'] = f1_per[i]
    return metrics

def train_combined(weight = True):
    print("="*60)
    print("COMBINED MULTILINGUAL TRAINING - SUBTASK 3")
    print("="*60)

    eng_train = pd.read_csv(os.path.join(workdir, "dev_phase/subtask3/train/eng.csv"))
    swa_train = pd.read_csv(os.path.join(workdir, "dev_phase/subtask3/train/swa.csv"))
    eng_dev = pd.read_csv(os.path.join(workdir, "dev_phase/subtask3/dev/eng.csv"))
    swa_dev = pd.read_csv(os.path.join(workdir, "dev_phase/subtask3/dev/swa.csv"))

    eng_train = augment_df(eng_train, 'eng', CONFIG['augment_factor'])
    swa_train = augment_df(swa_train, 'swa', CONFIG['augment_factor'])

    combined_train = pd.concat([eng_train, swa_train], ignore_index=True)
    print(f"\nCombined training: {len(combined_train)} samples")
    for label in LABELS:
        print(f"  {label}: {combined_train[label].sum()}")

    train_df, val_df = train_test_split(combined_train, test_size=0.15, random_state=42)

    pos_counts = train_df[LABELS].sum()
    neg_counts = len(train_df) - pos_counts
    class_weights = torch.clamp(torch.tensor(neg_counts / (pos_counts + 1e-6), dtype=torch.float32), 1.0, 10.0)

    tokenizer = AutoTokenizer.from_pretrained(CONFIG['model'])
    model = AutoModelForSequenceClassification.from_pretrained(CONFIG['model'], num_labels=6, problem_type="multi_label_classification")

    train_dataset = PolarizationDataset(train_df['text'].tolist(), train_df[LABELS].values.tolist(), tokenizer, CONFIG['max_len'])
    val_dataset = PolarizationDataset(val_df['text'].tolist(), val_df[LABELS].values.tolist(), tokenizer, CONFIG['max_len'])

    training_args = TrainingArguments(
        output_dir="./results_combined_subtask3",
        num_train_epochs=CONFIG['epochs'],
        learning_rate=CONFIG['lr'],
        per_device_train_batch_size=CONFIG['batch'],
        per_device_eval_batch_size=CONFIG['batch'] * 2,
        gradient_accumulation_steps=CONFIG['grad_accum'],
        warmup_ratio=CONFIG['warmup'],
        weight_decay=CONFIG['weight_decay'],
        eval_strategy="steps",
        eval_steps=100,
        save_strategy="steps",
        save_steps=100,
        load_best_model_at_end=True,
        metric_for_best_model="f1_macro",
        greater_is_better=True,
        logging_steps=50,
        fp16=True,
        report_to="none",
        save_total_limit=2,
    )
    if weight:
      trainer = WeightedTrainer(
          model=model,
          args=training_args,
          train_dataset=train_dataset,
          eval_dataset=val_dataset,
          compute_metrics=compute_metrics,
          data_collator=DataCollatorWithPadding(tokenizer),
          callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
          class_weights=class_weights
      )
    else:
      trainer = Trainer(
          model=model,
          args=training_args,
          train_dataset=train_dataset,
          eval_dataset=val_dataset,
          compute_metrics=compute_metrics,
          data_collator=DataCollatorWithPadding(tokenizer),
          callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
          class_weights=class_weights
      )

    trainer.train()

    final = trainer.evaluate()
    print(f"\nFinal F1 Macro: {final['eval_f1_macro']:.4f}")

    val_preds = trainer.predict(val_dataset)
    val_probs = torch.sigmoid(torch.tensor(val_preds.predictions)).numpy()
    best_thresh, best_f1 = 0.5, 0
    for thresh in np.arange(0.3, 0.55, 0.05):
        preds = (val_probs > thresh).astype(int)
        f1 = f1_score(val_df[LABELS].values, preds, average='macro', zero_division=0)
        if f1 > best_f1:
            best_f1, best_thresh = f1, thresh
    print(f"Best threshold: {best_thresh:.2f} (F1: {best_f1:.4f})")

    eng_dataset = PolarizationDataset(eng_dev['text'].tolist(), [[0]*6]*len(eng_dev), tokenizer, CONFIG['max_len'])
    eng_preds = trainer.predict(eng_dataset)
    eng_probs = torch.sigmoid(torch.tensor(eng_preds.predictions)).numpy()
    eng_binary = (eng_probs > best_thresh).astype(int)
    eng_result = pd.DataFrame(eng_binary, columns=LABELS)
    eng_result.insert(0, 'id', eng_dev['id'])
    eng_result.to_csv(os.path.join(workdir, "pred_eng_subtask3.csv"), index=False)
    print(f"\n✓ English predictions saved")
    print(f"Distribution:\n{eng_result[LABELS].sum()}")

    swa_dataset = PolarizationDataset(swa_dev['text'].tolist(), [[0]*6]*len(swa_dev), tokenizer, CONFIG['max_len'])
    swa_preds = trainer.predict(swa_dataset)
    swa_probs = torch.sigmoid(torch.tensor(swa_preds.predictions)).numpy()
    swa_binary = (swa_probs > best_thresh).astype(int)
    swa_result = pd.DataFrame(swa_binary, columns=LABELS)
    swa_result.insert(0, 'id', swa_dev['id'])
    swa_result.to_csv(os.path.join(workdir, "pred_swa_subtask3.csv"), index=False)
    print(f"\n✓ Swahili predictions saved")
    print(f"Distribution:\n{swa_result[LABELS].sum()}")



In [ ]:
train_combined(weight = True)

COMBINED MULTILINGUAL TRAINING - SUBTASK 3
Augmenting eng: 3222 samples
  Attempts: 3525, Successful changes: 430
  +430 augmented → 3652 total
Augmenting swa: 6991 samples
  Attempts: 10512, Successful changes: 1903
  +1903 augmented → 8894 total

Combined training: 12546 samples
  stereotype: 5234
  vilification: 5206
  dehumanization: 1714
  extreme_language: 3259
  lack_of_empathy: 3322
  invalidation: 3027


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

...

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,F1 Macro,F1 Stereotype,F1 Vilification,F1 Dehumanization,F1 Extreme Language,F1 Lack Of Empathy,F1 Invalidation
100,0.106900,0.098983,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
200,0.100900,0.097348,0.036510,0.219058,0.000000,0.000000,0.000000,0.000000,0.000000
300,0.098600,0.091921,0.033371,0.200225,0.000000,0.000000,0.000000,0.000000,0.000000
400,0.091900,0.091002,0.268818,0.575682,0.406812,0.251429,0.351906,0.027079,0.000000
500,0.088700,0.087738,0.349985,0.698917,0.493484,0.220736,0.175000,0.511771,0.000000
600,0.092200,0.085101,0.242415,0.647754,0.266811,0.292793,0.012220,0.234910,0.000000
700,0.090600,0.087042,0.450435,0.722110,0.558966,0.306306,0.324242,0.466346,0.324638
800,0.083200,0.086159,0.426425,0.706482,0.612274,0.308300,0.369958,0.410738,0.150794
900,0.086400,0.087090,0.493880,0.709815,0.615819,0.272109,0.464164,0.519481,0.381895
1000,0.081400,0.087837,0.525559,0.748731,0.661775,0.319290,0.383523,0.601307,0.438725



Final F1 Macro: 0.6334
Best threshold: 0.45 (F1: 0.6447)



✓ English predictions saved
Distribution:
stereotype          19
vilification        40
dehumanization      15
extreme_language    37
lack_of_empathy     18
invalidation        31
dtype: int64



✓ Swahili predictions saved
Distribution:
stereotype          149
vilification        161
dehumanization       65
extreme_language    102
lack_of_empathy     150
invalidation        118
dtype: int64
